本篇主要总结一些pythonic风格的列表推导以及代送的技巧，以达到节约内存和提高程序运行效率的目的

1. 列表推导式以及注意事项
2. 生成器
3. 代送技巧
4. Iterable与Iterator

# 列表推导

In [1]:
'列表推导'

# 基本用法：推导一个包含元素0-9的列表
a = [i for i in range(10)]
print(a)

# 根据a列表推导另一个列表
b = [i**2 for i in a]
print(b)

# 推导列表时，还可以增加条件
c = [i**2 for i in a if i % 2 == 0]
print(c)

## dict和set同样支持推导式
chile_rank = {'ghost':1, 'ha':2, 'ca':3}
rank_dick = {rank: name for name, rank in chile_rank.items()}
chile_len_set = {len(name) for name in rank_dick.values()}
print(rank_dick)
print(chile_len_set)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
[0, 4, 16, 36, 64]
{1: 'ghost', 2: 'ha', 3: 'ca'}
{2, 5}


列表推导注意事项：

1. 列表推导支持两个以上表达式的推导，但是尽量尽量别用
2. 列表太大的话，用推导式会很占用内存，影响程序效率

## 代送器

通过列表生成式，我们可以直接创建一个列表。但是，受到内存限制，列表容量肯定是有限的。如果我们创建列表的目的，是为了循环，当我们要循环非常多（比如几百万）元素时候，在循环前生成这个列表，就太占用内存了

我们仔细想想我们循环列表的过程。我们虽然要遍历列表中的每一个元素，但是每次循环所需的元素是非常有限的，对吧。所以，如果列表元素可以按照某种算法推算出来，那我们是否可以在循环的过程中，只生成和保留当前循环所需要的元素，循环过了而且不需要的元素直接删除，后续的元素后续再说。这样不就能不用创建完整的list，从而节省大量的空间。在Python中，这种一边循环一边计算的机制，称为生成器：generator

generator本身在python中就是一个对象，它保存了算法，然后每次代送时候计算出要用的值

使用generator也很简单，列表不是用[]嘛，改成()就行（不是元组，就是generator，元组是没有推导的）


In [18]:
d = (i for i in range(3))
d

<generator object <genexpr> at 0x000001778971C9E0>

In [19]:
# 可以使用next函数，改变代送器状态，生成新的元素
d = (i for i in range(3))
print(next(d))
print(next(d))

0
1


In [20]:
# generator本身也是可代送对象，自然可以用在for循环
d = (i for i in range(3))
for n in d:
    print(n)

0
1
2


In [21]:
# 再灵活点！用generator生成新generator
# 外围的generator每次前进，都会推动内部的generator，是的循环、评估条件等语句迅速执行
d = (i for i in range(3))
dd = ((i, i**2) for i in d)
dd

<generator object <genexpr> at 0x000001778971CCF0>

generator注意事项

1. 要非常注意，generator是有状态的！它只能按顺序调用，不能跳过也不能返回上一个
2. 同一个generator对象尽量不要反复使用


# 代送技巧
1. 单个列表+索引
2. 多列表

In [10]:
# 单列表代送时，若需要元素索引，使用enumerate()函数

name_list = ['张三','小明','老板']

for i, name in enumerate(name_list):
    print(f'第{i}个人是{name}')
                                                                                       

第0个人是张三
第1个人是小明
第2个人是老板


In [14]:
# 也可以指定开始计数时所用的值（不是从列表的第几个开始）
name_list = ['张三','小明','老板']
for i, name in enumerate(name_list,2):
    print(f'第{i}个人是{name}')

第2个人是张三
第3个人是小明
第4个人是老板


In [22]:
# enumerate不仅能用于列表，同样能用于generator！
d = (i for i in range(3))
for i,n in enumerate(d,2):
    print(i,n)

2 0
3 1
4 2


In [23]:
# 用zip()循环多个列表

name_list = ['张三','小明','老板']
pet_list = ['dog','cat', 'bird']

for name, pet in zip(name_list, pet_list):
    print(f'{name}养的宠物是{pet}')

张三养的宠物是dog
小明养的宠物是cat
老板养的宠物是bird


代送技巧注意事项：
1. zip()可以同时代送多个列表
2. zip()和enumerate()既能代送列表，也能代送generator
3. zip()在python3中相当于generator，在遍历过程中逐次产生元组
4. zip()代送次数，由其中最短的list或者generator长度决定，最短那个代送完了就结束了

# Iterable与Iterator

Iterable 可代送的，只要属于这个类型，都可以用for进行代送

Iterable包括：集合数据类型，如list、tuple、dict、set、str等；generator，包括生成器和带yield的generator function

isinstance()函数可以判断一个对象是否是Iterable

Iterable中有一类特殊的对象，是Iterator，也就是之前的generator。这些对象不但可以作用于for循环，还可以被next()函数不断调用并返回下一个值，直到最后抛出StopIteration错误表示无法继续返回下一个值了

可以使用isinstance()判断一个对象是否是Iterator对象

生成器都是Iterator对象，但list、dict、str虽然是Iterable，却不是Iterator

Iterator对象的本质，是一个有序序列，但我们却不能提前知道序列的长度，只能不断通过next()函数实现按需计算下一个数据，所以Iterator的计算是惰性的，只有在需要返回下一个数据时它才会计算。

Iterator甚至可以表示一个无限大的数据流，例如全体自然数。而使用list是永远不可能存储全体自然数的。